In [52]:
from math import sqrt
def distance(p1, p2):
    return sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 )

def get_speed(trip):
    speed = []
    for i in range(len(trip)-1):
        speed.append( distance(trip[i], trip[i+1])*3.6 )
    return speed

In [49]:
def get_trip(driver, index):
        with open('/Users/missoni/Documents/Kaggle/%d/%d.csv' % (driver,index)) as f:
            trip = []
            for line in f:
                try:
                    trip.append([float(i) for i in line.split(',')])
                except ValueError:
                    continue
        return trip

In [50]:
def plot_driver(driver, limit=200, color='b'):
    import matplotlib.pyplot as plt
    trips = range(1,limit+1)
    fig, ax = plt.subplots()

    for index in trips:
        trip = get_trip(driver, index)
        ax.plot([i[0] for i in trip],[i[1] for i in trip],'-', color='b')
            
    plt.show()

In [ ]:
plot_driver(1,10,'b')

Compute and plot speed over time, then compute acceleration

In [105]:
speed = get_speed(get_trip(1,2))

In [54]:
print len(speed)

862


In [64]:
def plot_speed(speed):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()

    ax.plot(range(len(speed)),speed,'-', color='b')
            
    plt.show()

In [68]:
%matplotlib 

Using matplotlib backend: MacOSX


In [141]:
plot_speed(speed)

In [71]:
from scipy import signal

In [73]:
filter_hz = 1e-3

In [74]:
sampling_frequency = 1

In [84]:
b, a = signal.butter(3, filter_hz / (sampling_frequency/2.), btype='low')

In [140]:
sf = signal.lfilter(b, a, speed)

No idea what a Butterworth filter does, but it has not helped.

In [79]:
def plot_filtered(before, after):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()

    ax.plot(range(len(before)),before,'-', color='b')
    ax.plot(range(len(after)),after,'-', color='r')
            
    plt.show()

In [126]:
plot_filtered(filtered_speed, speed)

Filter speed signal using a median filter. We want to get rid of noise, peaks and speed irregularities.

In [90]:
filtered_speed = signal.medfilt(speed, 9)

Smoothed filtered speed. Next step: select window 10 to 30 kmph.

In [127]:
def find_window_30(speed):
    
    begin = None
    begin_time = None
    for second, item in enumerate(speed):
        if item > 10:
            continue
        begin = item
        begin_time = second
        break
        
    print 'speed', begin, 'at', begin_time, 'sec'
    
    for second, item in enumerate(speed[begin+1:], start=begin_time+1):    
        if item > 10:
            begin = item
            begin_time = second
            break
    
    for second, item in enumerate(speed[begin+1:], start=begin_time+1):    
        if item > 30:
            end = item
            end_time = second
            break
        
    print 'speed', begin, 'at', begin_time, 'sec', end, 'at', end_time

In [138]:
find_window_30(filtered_speed)

speed 8.08999381953 at 6 sec
speed 13.6989342651 at 12 sec 32.096579257 at 22


Here, we have computed the interval where the driver's speed exceeded 10 kmph and achieved 30 kmph. This is a 10-to-30 speed window. Next, we want to measure the acceleration.

In [133]:
def get_acceleration(speed_interval):
    acc = []
    for i in range(len(speed_interval)-1):
        acc.append(speed_interval[i+1] - speed_interval[i])
    return acc


In [139]:
print get_acceleration(filtered_speed[14:24])


[2.017859301001879, 3.8759397807113682, 3.6851016040407849, 0.40567830202271793, 1.3927544119720459, 0.085131342343483851, 3.0796774595260636, 1.1865839907320641, 2.668918799518714]


This is the acceleration in the first 10-to-30 interval.

In [136]:
def plot_acc(acc):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()

    ax.plot(range(len(acc)),acc,'-', color='b')
            
    plt.show()

In [137]:
plot_acc(get_acceleration(filtered_speed[14:24]))

Next step will be to analyze acceleration across all 10-to-30 intervals and compute some scalar metric, e.g. acceleration time. Then, compute the same metric for other speed intervals, 50 to 70, 70 to 100, 100 to 120 and above.